In [8]:
import pennylane.numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import pennylane as qml
from pennylane.optimize import AdamOptimizer

# Attempt 1 : Classifier with 4 features standard  optimsier 

In [9]:


# Load Breast Cancer Dataset
data = load_breast_cancer()
X, y = data.data, data.target

# Standardize Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reduce to 4 Features Using PCA
pca = PCA(n_components=4)
X_pca = pca.fit_transform(X_scaled)

# Train-Test Split (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Function to Normalize Data for Amplitude Encoding
def normalize_features(features):
    return features / np.linalg.norm(features, axis=1, keepdims=True)

# Normalize Train and Test Features
X_train_norm = normalize_features(X_train)
X_test_norm = normalize_features(X_test)

# Quantum Circuit Setup
n_qubits = 2  # 4 features => log2(4) = 2 qubits
dev = qml.device("default.qubit", wires=n_qubits)

def quantum_circuit(weights, x):
    """Amplitude encoding and a variational ansatz."""
    qml.AmplitudeEmbedding(features=x, wires=range(n_qubits), normalize=False)
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))

# Initialize weights for variational circuit
num_layers = 3
weights_shape = (num_layers, n_qubits)

weights = np.random.randn(*weights_shape, requires_grad=True)


@qml.qnode(dev)
def circuit(weights, x):
    return quantum_circuit(weights, x)

# Define Cost Function
def cost(weights, X, y):
    predictions = np.array([circuit(weights, x) for x in X])
    return np.mean((predictions - y) ** 2)

# Train Quantum Model
opt = AdamOptimizer(stepsize=0.1)
steps = 100

for step in range(steps):
    weights, cost_value = opt.step_and_cost(lambda w: cost(w, X_train_norm, y_train), weights)
    if step % 10 == 0:
        print(f"Step {step}: Cost = {cost_value}")

# Evaluate Quantum Model
y_pred_quantum = [1 if circuit(weights, x) > 0.5 else 0 for x in X_test_norm]
quantum_accuracy = accuracy_score(y_test, y_pred_quantum)
print(f"Quantum Classifier Accuracy: {quantum_accuracy}")
print(classification_report(y_test, y_pred_quantum))

# Train Logistic Regression Model
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

# Evaluate Logistic Regression Model
y_pred_logistic = log_reg.predict(X_test)
logistic_accuracy = accuracy_score(y_test, y_pred_logistic)
print(f"Logistic Regression Accuracy: {logistic_accuracy}")
print(classification_report(y_test, y_pred_logistic))


Step 0: Cost = 0.8939561639313268
Step 10: Cost = 0.4641770794285326
Step 20: Cost = 0.44398438527110257
Step 30: Cost = 0.44807538496965565
Step 40: Cost = 0.44192196362218084
Step 50: Cost = 0.442238239583276
Step 60: Cost = 0.44209399560795715
Step 70: Cost = 0.44183685992717125
Step 80: Cost = 0.441787158212142
Step 90: Cost = 0.44179068608812494
Quantum Classifier Accuracy: 0.4444444444444444
              precision    recall  f1-score   support

           0       0.36      0.68      0.48        63
           1       0.62      0.31      0.41       108

    accuracy                           0.44       171
   macro avg       0.49      0.49      0.44       171
weighted avg       0.53      0.44      0.43       171

Logistic Regression Accuracy: 0.9824561403508771
              precision    recall  f1-score   support

           0       0.98      0.97      0.98        63
           1       0.98      0.99      0.99       108

    accuracy                           0.98       171
   ma

In [10]:


# Load Breast Cancer Dataset
data = load_breast_cancer()
X, y = data.data, data.target

# Standardize Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reduce to 4 Features Using PCA
pca = PCA(n_components=4)
X_pca = pca.fit_transform(X_scaled)

# Train-Test Split (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Function to Normalize Data for Amplitude Encoding
def normalize_features(features):
    return features / np.linalg.norm(features, axis=1, keepdims=True)

# Normalize Train and Test Features
X_train_norm = normalize_features(X_train)
X_test_norm = normalize_features(X_test)

# Quantum Circuit Setup
n_qubits = 2  # 4 features => log2(4) = 2 qubits
dev = qml.device("default.qubit", wires=n_qubits)

def quantum_circuit(weights, x):
    """Amplitude encoding and a variational ansatz."""
    qml.AmplitudeEmbedding(features=x, wires=range(n_qubits), normalize=False)
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))

# Initialize weights for variational circuit
num_layers = 6  # Increased the number of layers
weights_shape = (num_layers, n_qubits, 3)  # StronglyEntanglingLayers requires 3 parameters per qubit
weights = np.random.randn(*weights_shape, requires_grad=True)

@qml.qnode(dev)
def circuit(weights, x):
    return quantum_circuit(weights, x)

# Define Cost Function
def cost(weights, X, y):
    predictions = np.array([circuit(weights, x) for x in X])
    return np.mean((predictions - y) ** 2)

# Train Quantum Model
opt = qml.AdamOptimizer(stepsize=0.1)  # Using Adam optimizer
steps = 100

for step in range(steps):
    weights, cost_value = opt.step_and_cost(lambda w: cost(w, X_train_norm, y_train), weights)
    if step % 10 == 0:
        print(f"Step {step}: Cost = {cost_value}")

# Evaluate Quantum Model
y_pred_quantum = [1 if circuit(weights, x) > 0.5 else 0 for x in X_test_norm]
quantum_accuracy = accuracy_score(y_test, y_pred_quantum)
print(f"Quantum Classifier Accuracy: {quantum_accuracy}")
print(classification_report(y_test, y_pred_quantum))

# Train Logistic Regression Model
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

# Evaluate Logistic Regression Model
y_pred_logistic = log_reg.predict(X_test)
logistic_accuracy = accuracy_score(y_test, y_pred_logistic)
print(f"Logistic Regression Accuracy: {logistic_accuracy}")
print(classification_report(y_test, y_pred_logistic))


Step 0: Cost = 1.0681601063932151
Step 10: Cost = 0.4569216215170072
Step 20: Cost = 0.40684127341311915
Step 30: Cost = 0.37532742321522333
Step 40: Cost = 0.36199131161999304
Step 50: Cost = 0.35892386699991613
Step 60: Cost = 0.35843796379427845
Step 70: Cost = 0.3580776004657273
Step 80: Cost = 0.3580906442799102
Step 90: Cost = 0.3580304489848019
Quantum Classifier Accuracy: 0.5087719298245614
              precision    recall  f1-score   support

           0       0.40      0.65      0.49        63
           1       0.68      0.43      0.52       108

    accuracy                           0.51       171
   macro avg       0.54      0.54      0.51       171
weighted avg       0.57      0.51      0.51       171

Logistic Regression Accuracy: 0.9824561403508771
              precision    recall  f1-score   support

           0       0.98      0.97      0.98        63
           1       0.98      0.99      0.99       108

    accuracy                           0.98       171
   m